In [113]:
import pandas as pd

In [114]:
## 파일 로드
uriage = pd.read_csv("../csv/uriage.csv")
uriage.head()  # 데이터가 문제가 많은 것을 볼 수 있음..결측치도 있고..item name 도 대/소문자..빈칸들도 있고 없고...

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [115]:
## 결측치 확인
uriage.isna().sum() # sum() 을 빼면 결측치 가 있는 것을 다 볼 수 있고 .sum() 을 하면 어디에 얼마나 결측치가 있는지 한눈에 봄

purchase_date      0
item_name          0
item_price       387
customer_name      0
dtype: int64

In [116]:
uriage.info() # 데이터 확인 가능 . purchase_date 가 시계열이 아닌 문자열("object") 로 되어있다고 확인 가능함

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   purchase_date  2999 non-null   object 
 1   item_name      2999 non-null   object 
 2   item_price     2612 non-null   float64
 3   customer_name  2999 non-null   object 
dtypes: float64(1), object(3)
memory usage: 93.8+ KB


In [117]:
# 1. purchase_date 컬럼의 데이터 타입을 시계열 데이터로 변경
# 2. item_name 의 데이터를 모두 대문자로 변경
# 3. item_name 의 데이터에서 공백을 삭제
# 4. item_name 을 기준으로 오름차순 정렬
# 5. 인덱스를 초기화(기존의 인덱스는 삭제)

In [118]:
## 1. 시계열 데이터로 변경
## pd.to_datetime(변경할 데이터, format='데이터의 형태') 함수 사용하여 데이터를 시계열로 변경
## pd.to_datetime(uriage['purchase_date'],
               # format = '%Y-%m-%d %H:%M')  # 원본의 purchase_date 의 형태 그대로 적어줘야 됨. 뛰우기, '-' 등 모두..
               # 대문자 Y 는 네자리 연도수, 소문자 y 는 두자리 연도수 형태
               # H 대문자는 24시간 소문자는 12시간 형태
               # 이것은 실행이 아니고 확인하는 것
uriage['purchase_date'] = pd.to_datetime(uriage['purchase_date'],
               format = '%Y-%m-%d %H:%M')

In [119]:
uriage.info() # 아래에서 datetime64 형태로 변경된 것을 볼 수 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   purchase_date  2999 non-null   datetime64[ns]
 1   item_name      2999 non-null   object        
 2   item_price     2612 non-null   float64       
 3   customer_name  2999 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 93.8+ KB


In [120]:
## 2. item_name 칼럼의 데이터를 대문자로 변경
## uriage['item_name'].upper() # upper 는 그냥 문자열 하나하나 에는 사용가능하나 스리즈 전체를 다 적용하는 것에는 에러가 남 그래서..
                               # 아래의 각 case 로...
# case. 1
uriage['item_name'].str.upper() # 스리즈 안에 있는 문자열을 바꾼다는 표시

# case. 2
# apply() : 스리즈 형태의 데이터를 항목 하나하나 씩 함수에 대입하여 출력값을 스리즈의 형태로 변경하여 출력하는 함수
           # apply 함수안에 함수를 호출해줘야 함

## 일반 함수 생성
def change(x):
    return x.upper()

uriage['item_name'].apply(change) # 함수를 호출할 때는 인자를 넣어줘야 하는데..change 에 인자를 안넣은 것은 apply 가 인자를 적용해주는 함수

# case. 3 : lambda 함수를 apply 안에 사용하여 위의 case 2 를 한줄로 간단히
## apply(lambda 함수)
uriage['item_name'].apply(lambda x : x.upper())

## 위의 것 모두 결과만 조회한 것이고 data 가 바뀌진 않았음

0         상품A
1       상 품 S
2       상 품 A
3         상품Z
4         상품A
        ...  
2994      상품Y
2995      상품M
2996      상품Q
2997      상품H
2998      상품D
Name: item_name, Length: 2999, dtype: object

In [121]:
## 공백제거
## 문자열 사이의 공백도 삭제해야되기 때문에 replace(" ","") 사용해야 함. 

# case. 1
uriage['item_name'].str.replace(" ","")

# case. 2
def change(x):
    return x.replace(" ","")

uriage['item_name'].apply(change)

# case. 3

uriage['item_name'].apply(lambda x : x.replace(" ",""))

## 위의 것 모두 결과만 조회한 것이고 data 가 바뀌진 않았음

0       상품A
1       상품S
2       상품a
3       상품Z
4       상품a
       ... 
2994    상품Y
2995    상품M
2996    상품Q
2997    상품H
2998    상품D
Name: item_name, Length: 2999, dtype: object

In [122]:
## 대문자로 변경하고 공백제거 2개의 행동을 한번에 하는 방법(내가 한 방법)

## uriage['item_name'] = uriage['item_name'].apply(lambda x : x.upper())   
## uriage['item_name'] = uriage['item_name'].apply(lambda x : x.replace(" ",""))

In [123]:
## 대문자로 변경하고 공백제거 2개의 행동을 한번에 하는 방법(강사 방법)
# case 1
uriage['item_name'].str.upper().str.replace(" ","") # 실행 결과를 보여주기만 한 것
# case 2
def change(x):
    # result = x.upper().replace(" ","") 또는
    result = x.upper()
    result = result.replace(" ","")
    return result

uriage['item_name'].apply(change)
# case 3
uriage['item_name'].apply(lambda x : x.upper().replace(" ","")) # upper replace 를 따로 쓰려면 lambda 를 두번 써야 함)

0       상품A
1       상품S
2       상품A
3       상품Z
4       상품A
       ... 
2994    상품Y
2995    상품M
2996    상품Q
2997    상품H
2998    상품D
Name: item_name, Length: 2999, dtype: object

In [124]:
uriage['item_name'] = uriage['item_name'].apply(lambda x : x.upper().replace(" ","")) # 위의 방법으로 이제 data 를 바꿔버린 것

In [125]:
# 4. item_name 을 기준으로 오름차순 정렬로 변경
uriage.sort_values(['item_name'], inplace=True) # 오름차순이 기본이라 굳이 ascending 안적어줘도 됨


In [126]:
# 5. 인덱스를 초기화(기존의 인덱스는 삭제)
uriage.reset_index(drop=True, inplace = True) # inplace = True 를 빼고 먼저 확인하는 것이 좋음

In [127]:
# 결측치에 데이터를 치환
# 1. 상품A 의 item_price 의 값은 결측치 중 상품이 A 이면 상품A 의 평균 값을 치환(현재 DATA 에서는 상품 A 의 값이 모두 동일)
# 2. 나머지 상품들도 같은 값으로 치환
# 상품A 부터 Z 까지 평균값을 먼저 다 구해야하므로 같은 작업을 반복하는 반복문도 사용해야함

In [128]:
## Item_name 이 상품A 인 경우만 추출

uriage['item_name'] == '상품A'

0        True
1        True
2        True
3        True
4        True
        ...  
2994    False
2995    False
2996    False
2997    False
2998    False
Name: item_name, Length: 2999, dtype: bool

In [129]:
fig = uriage['item_name'] == '상품A'

In [130]:
uriage.loc[fig] # 상품이 A 인 것의 Location 만 보여주는 것

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:00,상품A,100.0,김가온
1,2019-05-19 20:22:00,상품A,100.0,김시훈
2,2019-06-25 08:13:00,상품A,100.0,김유진
3,2019-06-13 16:03:00,상품A,100.0,김건희
4,2019-02-10 00:28:00,상품A,100.0,김하랑
...,...,...,...,...
134,2019-01-13 11:50:00,상품A,100.0,김시율
135,2019-03-09 14:20:00,상품A,NaN,김우찬
136,2019-05-11 19:42:00,상품A,NaN,김유찬
137,2019-04-25 17:15:00,상품A,100.0,김강현


In [131]:
uriage.loc[fig,'item_price'].isna() # 상품A 이고 결측치 item_price 가 결측치인 경우 True

0      False
1      False
2      False
3      False
4      False
       ...  
134    False
135     True
136     True
137    False
138    False
Name: item_price, Length: 139, dtype: bool

In [132]:
fig2 = uriage.loc[fig,'item_price'].isna() # 상품A 이고 결측치 item_price 가 결측치인 경우 True 인것만 fig2 라는 변수값을 만들어주고..

In [133]:
uriage.loc[fig].loc[fig2] # fig 에서 fig2 만 보여주는 것

,purchase_date,item_name,item_price,customer_name
18,2019-04-11 02:22:00,상품A,NaN,김선우
22,2019-07-07 21:49:00,상품A,NaN,김은우
32,2019-07-14 03:39:00,상품A,NaN,김시우
38,2019-05-25 12:51:00,상품A,NaN,김재민
44,2019-02-28 04:11:00,상품A,NaN,김서준
46,2019-02-13 07:28:00,상품A,NaN,김시환
48,2019-07-31 01:32:00,상품A,NaN,김윤찬
51,2019-04-06 11:06:00,상품A,NaN,김태인
58,2019-03-15 09:14:00,상품A,NaN,김강현
59,2019-06-24 05:06:00,상품A,NaN,김시호


In [134]:
## 조건식 2개 
fig = uriage['item_name'] =='상품A'
fig2 = uriage['item_price'].isna()

In [135]:
uriage.loc[fig & fig2] # 상품이 a 이고 결측이의 값만 나옴. 위의 조건식 두개를 같이...
uriage.loc[fig & ~fig2] # ~ 를 넣어주면 반대로 결측이 아닌 값만 나옴

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:00,상품A,100.0,김가온
1,2019-05-19 20:22:00,상품A,100.0,김시훈
2,2019-06-25 08:13:00,상품A,100.0,김유진
3,2019-06-13 16:03:00,상품A,100.0,김건희
4,2019-02-10 00:28:00,상품A,100.0,김하랑
...,...,...,...,...
131,2019-06-05 02:25:00,상품A,100.0,김현성
133,2019-06-24 02:56:00,상품A,100.0,김현진
134,2019-01-13 11:50:00,상품A,100.0,김시율
137,2019-04-25 17:15:00,상품A,100.0,김강현


In [136]:
## 상품이 A 인 경우 & 결측치가 아닌 경우에 item_price의 평균값을 구함
a_price = uriage.loc[fig & ~ fig2, 'item_price'].mean()
a_price

100.0

In [137]:
## 상품A 이고 결측치인 item_price 가 결측치인 값에 치환
uriage.loc[fig & fig2].fillna(a_price) # fig 는 상품이 A 인 것 fig2 는 A 중 price가 결측치 만인 것

,purchase_date,item_name,item_price,customer_name
18,2019-04-11 02:22:00,상품A,100.0,김선우
22,2019-07-07 21:49:00,상품A,100.0,김은우
32,2019-07-14 03:39:00,상품A,100.0,김시우
38,2019-05-25 12:51:00,상품A,100.0,김재민
44,2019-02-28 04:11:00,상품A,100.0,김서준
46,2019-02-13 07:28:00,상품A,100.0,김시환
48,2019-07-31 01:32:00,상품A,100.0,김윤찬
51,2019-04-06 11:06:00,상품A,100.0,김태인
58,2019-03-15 09:14:00,상품A,100.0,김강현
59,2019-06-24 05:06:00,상품A,100.0,김시호


In [138]:
uriage.value_counts('item_name') # 상품이 어떤 것이 있는지 확인할 수 있음
# uriage.value_counts('item_name').index() 하면 더 간단함..각 상품의 건수는 빼고 상품 종류만 보여 줌

item_name
상품S    140
상품A    139
상품R    136
상품P    135
상품V    134
상품C    133
상품F    126
상품D    125
상품Q    125
상품O    124
상품W    121
상품N    119
상품T    117
상품J    116
상품X    116
상품I    114
상품M    113
상품B    113
상품L    110
상품G    110
상품K    109
상품H    109
상품U    107
상품E    105
상품Y    102
상품Z      1
dtype: int64

In [139]:
uriage['item_name'].unique()

array(['상품A', '상품B', '상품C', '상품D', '상품E', '상품F', '상품G', '상품H', '상품I',
       '상품J', '상품K', '상품L', '상품M', '상품N', '상품O', '상품P', '상품Q', '상품R',
       '상품S', '상품T', '상품U', '상품V', '상품W', '상품X', '상품Y', '상품Z'],
      dtype=object)

In [140]:
name_list = uriage['item_name'].unique() # name_list 라는 변수를 만들어 위의 결과값을 가지고 있게 함

In [141]:
## 반복문을 이용하여 모든 결측치를 치환
uriage2 = uriage.copy()

In [142]:
for i in name_list:
    print(i)

상품A
상품B
상품C
상품D
상품E
상품F
상품G
상품H
상품I
상품J
상품K
상품L
상품M
상품N
상품O
상품P
상품Q
상품R
상품S
상품T
상품U
상품V
상품W
상품X
상품Y
상품Z


In [143]:
for i in name_list:
    # 조건식 생성
    fig = uriage['item_name'] == i
    fig2 = uriage['item_price'].isna()

    # fig 와 ~fig2 의 조건식이 모두 참일때의 item_price 평균 값
    mean_price = uriage.loc[fig & ~fig2, 'item_price'].mean()
    # fig 와 fig2 의 ㅣ조건식이 모두 참인 경우 item_price 에 mean_price 의 값으로 대체
    uriage.loc[fig & fig2, 'item_price'] = mean_price # loc[fig & fig2, 'item_price'] 는 결측치만 찾은 것

In [144]:
uriage.isna().sum() # 결측치가 다 사라진 것을 볼 수 있음

purchase_date    0
item_name        0
item_price       0
customer_name    0
dtype: int64

In [145]:
# 함수를 만들때는 어떤 것을 고정변수로 어떤 것을 가변 변수로 할 것인지 먼저 고민해야 함..

def change(df, x): # 위의 DataFrame(uriage) 을 변수로 할 것이라 생각하여 df 로 상징적으로 사용, item_name 은 고정변수로 결정 함
    ## 조건식 생성
    fig = df['item_name'] == x # x 변수를 for 문의 i 로 사용 하기 때문...
    fig2 = df['item_price'].isna()

    ## fig 그리고 ~fig 인경우 item_price 의 평균
    mean_price = df.loc[fig & ~fig2, 'item_price'].mean()

    ## 평균값을 결측치에 대체
    df.loc[fig & fig2, 'item_price'] = mean_price

In [146]:
# 위와 같이 함수를 만들고 for 문을 만들면 됨. 위의 함수에 return 을 안만든 것은 df 이 어짜피 계속 바뀔 것이기 때문에라고 함....

In [149]:
uriage2.isna().sum() # 결측치 있는 것 볼 수 있음

purchase_date      0
item_name          0
item_price       387
customer_name      0
dtype: int64

In [150]:
for i in name_list:
  change(uriage2,i)

In [151]:
uriage2.isna().sum() # 결측치 사라진 것 볼 수 있음

purchase_date    0
item_name        0
item_price       0
customer_name    0
dtype: int64

In [153]:
uriage.query("item_name == '상품B'") # query 함수도 ()에 조건식을 적어줌. "" 는 확실히 두대있는 따움표로 다 묶어줘야 됨(한개 아님)

,purchase_date,item_name,item_price,customer_name
139,2019-06-07 00:03:00,상품B,200.0,김재희
140,2019-03-15 06:06:00,상품B,200.0,김서윤
141,2019-06-25 03:12:00,상품B,200.0,김규민
142,2019-04-18 16:44:00,상품B,200.0,김유빈
143,2019-07-27 09:00:00,상품B,200.0,김현수
...,...,...,...,...
247,2019-02-17 01:46:00,상품B,200.0,김지완
248,2019-05-21 11:17:00,상품B,200.0,김태우
249,2019-07-02 07:39:00,상품B,200.0,김정민
250,2019-07-09 15:27:00,상품B,200.0,김성민
